In [1]:
# !pip install torcheval

from torcheval.metrics.functional import binary_auroc

# Import Libraries

In [2]:
import os
import gc
import math
import copy
import time
import random
import glob
import timm
import cv2
import h5py

from matplotlib import pyplot as plt
from pathlib import Path

import numpy as np
import pandas as pd
import polars as pl
from PIL import Image
from io import BytesIO

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.cuda import amp

from sklearn.model_selection import StratifiedKFold, StratifiedGroupKFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import VotingClassifier

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.pipeline import Pipeline

# import lightgbm as lgb
# import catboost as cb
# import xgboost as xgb

from sklearn.utils import resample

import joblib
from tqdm import tqdm
from collections import defaultdict

import albumentations as A
from albumentations.pytorch import ToTensorV2

import optuna

/home/kist/anaconda3/envs/kk/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Setting Seed

In [3]:
def set_seed(seed=42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    
set_seed()

In [4]:
# For colored terminal text
from colorama import Fore, Back, Style
b_ = Fore.BLUE
sr_ = Style.RESET_ALL

import warnings
warnings.filterwarnings("ignore")

# For descriptive error messages
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [5]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# Load Train Dataset

In [6]:
root_dir = "data/isic-skin-cancer"
train_image_dir = f'{root_dir}/train-image/image'

In [7]:
def get_train_file_path(image_id):
    return f"{train_image_dir}/{image_id}.jpg"

In [8]:
train_images = sorted(glob.glob(f"{train_image_dir}/*.jpg"))

## Load Extra Train Dataset

In [9]:
train_2018_image_dir = "data/isic-2018/train-image/image"
train_2019_image_dir = "data/isic-2019/train-image/image"
train_2020_image_dir = "data/isic-2020/train-image/image"

In [10]:
def get_2018_train_file_path(image_id):
    return f"{train_2018_image_dir}/{image_id}.jpg"

def get_2019_train_file_path(image_id):
    return f"{train_2019_image_dir}/{image_id}.jpg"

def get_2020_train_file_path(image_id):
    return f"{train_2020_image_dir}/{image_id}.jpg"


In [11]:
train_2018_images = sorted(glob.glob(f"{train_2018_image_dir}/*.jpg"))
train_2019_images = sorted(glob.glob(f"{train_2019_image_dir}/*.jpg"))
train_2020_images = sorted(glob.glob(f"{train_2020_image_dir}/*.jpg"))

## Reduce Data Imabalance

In [12]:
image_df = pd.read_csv(f"{root_dir}/train-metadata.csv")

print("df.shape, # of positive cases, # of patients")
print("original>", image_df.shape, image_df.target.sum(), image_df["patient_id"].unique().shape)

image_df_postive = image_df[image_df["target"] == 1].reset_index(drop=True)
image_df_negative = image_df[image_df["target"] == 0].reset_index(drop=True)

image_df = pd.concat([image_df_postive, image_df_negative.iloc[:image_df_postive.shape[0]*20, :]])
print("filtered>", image_df.shape, image_df.target.sum(), image_df["patient_id"].unique().shape)

image_df['file_path'] = image_df['isic_id'].apply(get_train_file_path)
image_df = image_df[ image_df["file_path"].isin(train_images) ].reset_index(drop=True)
image_df = image_df[['isic_id', 'target', 'patient_id', 'file_path']]
print(image_df.isnull().sum())
image_df.head()

df.shape, # of positive cases, # of patients
original> (401059, 55) 393 (1042,)
filtered> (8253, 55) 393 (950,)
isic_id       0
target        0
patient_id    0
file_path     0
dtype: int64


,isic_id,target,patient_id,file_path
0,ISIC_0082829,1,IP_3249371,data/isic-skin-cancer/train-image/image/ISIC_0...
1,ISIC_0096034,1,IP_6723298,data/isic-skin-cancer/train-image/image/ISIC_0...
2,ISIC_0104229,1,IP_9057861,data/isic-skin-cancer/train-image/image/ISIC_0...
3,ISIC_0119495,1,IP_6856511,data/isic-skin-cancer/train-image/image/ISIC_0...
4,ISIC_0157834,1,IP_3927284,data/isic-skin-cancer/train-image/image/ISIC_0...


### Reduce Data Imbalance - Extra Dataset (2018)

In [13]:
image_2018_df = pd.read_csv("data/isic-2018/train-metadata.csv")

print("df.shape, # of positive cases, # of patients")
print("original>", image_2018_df.shape, image_2018_df.target.sum(), image_2018_df["patient_id"].unique().shape)

image_2018_df_postive = image_2018_df[image_2018_df["target"] == 1].reset_index(drop=True)
image_2018_df_negative = image_2018_df[image_2018_df["target"] == 0].reset_index(drop=True)

print(image_2018_df_postive.shape, image_2018_df_negative.shape)

image_2018_df = pd.concat([image_2018_df_postive, image_2018_df_negative.iloc[:image_2018_df_postive.shape[0]*20, :]])
print("filtered>", image_2018_df.shape, image_2018_df.target.sum(), image_2018_df["patient_id"].unique().shape)

image_2018_df['file_path'] = image_2018_df['isic_id'].apply(get_2018_train_file_path)
image_2018_df = image_2018_df[ image_2018_df["file_path"].isin(train_2018_images) ].reset_index(drop=True)
image_2018_df = image_2018_df.drop('Unnamed: 0', axis=1)
print(image_2018_df.columns)
print(image_2018_df.isnull().sum())
image_2018_df.head()

df.shape, # of positive cases, # of patients
original> (8982, 4) 785.0 (8982,)
(785, 4) (8197, 4)
filtered> (8982, 4) 785.0 (8982,)
Index(['isic_id', 'patient_id', 'target', 'file_path'], dtype='object')
isic_id       0
patient_id    0
target        0
file_path     0
dtype: int64


,isic_id,patient_id,target,file_path
0,ISIC_0034246,HAM_0000002,1.0,data/isic-2018/train-image/image/ISIC_0034246.jpg
1,ISIC_0034162,HAM_0000006,1.0,data/isic-2018/train-image/image/ISIC_0034162.jpg
2,ISIC_0031377,HAM_0000022,1.0,data/isic-2018/train-image/image/ISIC_0031377.jpg
3,ISIC_0027190,HAM_0000040,1.0,data/isic-2018/train-image/image/ISIC_0027190.jpg
4,ISIC_0034048,HAM_0000042,1.0,data/isic-2018/train-image/image/ISIC_0034048.jpg


### Reduce Data Imbalance - Extra Dataset (2019)

In [14]:
image_2019_df = pd.read_csv("data/isic-2019/train-metadata.csv")

print("df.shape, # of positive cases, # of patients")
print("original>", image_2019_df.shape, image_2019_df.target.sum(), image_2019_df["patient_id"].unique().shape)

image_2019_df_postive = image_2019_df[image_2019_df["target"] == 1].reset_index(drop=True)
image_2019_df_negative = image_2019_df[image_2019_df["target"] == 0].reset_index(drop=True)

print(image_2019_df_postive.shape, image_2019_df_negative.shape)

image_2019_df = pd.concat([image_2019_df_postive, image_2019_df_negative.iloc[:image_2019_df_postive.shape[0]*20, :]])
print("filtered>", image_2019_df.shape, image_2019_df.target.sum(), image_2019_df["patient_id"].unique().shape)

image_2019_df['file_path'] = image_2019_df['isic_id'].apply(get_2019_train_file_path)
image_2019_df = image_2019_df[ image_2019_df["file_path"].isin(train_2019_images) ].reset_index(drop=True)
image_2019_df = image_2019_df.drop('Unnamed: 0', axis=1)
print(image_2019_df.columns)
print(image_2019_df.isnull().sum())
image_2019_df.head()

df.shape, # of positive cases, # of patients
original> (25331, 4) 4522 (25331,)
(4522, 4) (20809, 4)
filtered> (25331, 4) 4522 (25331,)
Index(['isic_id', 'patient_id', 'target', 'file_path'], dtype='object')
isic_id       0
patient_id    0
target        0
file_path     0
dtype: int64


,isic_id,patient_id,target,file_path
0,ISIC_0000002,dummy_2,1,data/isic-2019/train-image/image/ISIC_0000002.jpg
1,ISIC_0000004,dummy_4,1,data/isic-2019/train-image/image/ISIC_0000004.jpg
2,ISIC_0000013,dummy_12,1,data/isic-2019/train-image/image/ISIC_0000013.jpg
3,ISIC_0000022_downsampled,dummy_21,1,data/isic-2019/train-image/image/ISIC_0000022_...
4,ISIC_0000026_downsampled,dummy_25,1,data/isic-2019/train-image/image/ISIC_0000026_...


### Reduce Data Imbalance - Extra Dataset (2020)

In [15]:
image_2020_df = pd.read_csv("data/isic-2020/train-metadata.csv")

print("df.shape, # of positive cases, # of patients")
print("original>", image_2020_df.shape, image_2020_df.target.sum(), image_2020_df["patient_id"].unique().shape)

image_2020_df_postive = image_2020_df[image_2020_df["target"] == 1].reset_index(drop=True)
image_2020_df_negative = image_2020_df[image_2020_df["target"] == 0].reset_index(drop=True)

print(image_2020_df_postive.shape, image_2020_df_negative.shape)

image_2020_df = pd.concat([image_2020_df_postive, image_2020_df_negative.iloc[:image_2020_df_postive.shape[0]*20, :]])
print("filtered>", image_2020_df.shape, image_2020_df.target.sum(), image_2020_df["patient_id"].unique().shape)

image_2020_df['file_path'] = image_2020_df['isic_id'].apply(get_2020_train_file_path)
image_2020_df = image_2020_df[ image_2020_df["file_path"].isin(train_2020_images) ].reset_index(drop=True)
image_2020_df = image_2020_df.drop('Unnamed: 0', axis=1)
print(image_2020_df.columns)
print(image_2020_df.isnull().sum())
image_2020_df.head()

df.shape, # of positive cases, # of patients
original> (33126, 4) 584 (2056,)
(584, 4) (32542, 4)
filtered> (12264, 4) 584 (1951,)
Index(['isic_id', 'patient_id', 'target', 'file_path'], dtype='object')
isic_id       0
patient_id    0
target        0
file_path     0
dtype: int64


,isic_id,patient_id,target,file_path
0,ISIC_0149568,IP_0962375,1,data/isic-2020/train-image/image/ISIC_0149568.jpg
1,ISIC_0188432,IP_0135517,1,data/isic-2020/train-image/image/ISIC_0188432.jpg
2,ISIC_0207268,IP_7735373,1,data/isic-2020/train-image/image/ISIC_0207268.jpg
3,ISIC_0232101,IP_8349964,1,data/isic-2020/train-image/image/ISIC_0232101.jpg
4,ISIC_0247330,IP_3232631,1,data/isic-2020/train-image/image/ISIC_0247330.jpg


In [16]:
combined_df = pd.concat([image_df, image_2018_df, image_2019_df, image_2020_df], axis=0, ignore_index=True)
print(combined_df.shape)
combined_df

(53344, 4)


,isic_id,target,patient_id,file_path
0,ISIC_0082829,1.0,IP_3249371,data/isic-skin-cancer/train-image/image/ISIC_0...
1,ISIC_0096034,1.0,IP_6723298,data/isic-skin-cancer/train-image/image/ISIC_0...
2,ISIC_0104229,1.0,IP_9057861,data/isic-skin-cancer/train-image/image/ISIC_0...
3,ISIC_0119495,1.0,IP_6856511,data/isic-skin-cancer/train-image/image/ISIC_0...
4,ISIC_0157834,1.0,IP_3927284,data/isic-skin-cancer/train-image/image/ISIC_0...
...,...,...,...,...
53339,ISIC_3661660,0.0,IP_7610036,data/isic-2020/train-image/image/ISIC_3661660.jpg
53340,ISIC_3662337,0.0,IP_4021847,data/isic-2020/train-image/image/ISIC_3662337.jpg
53341,ISIC_3662678,0.0,IP_6245507,data/isic-2020/train-image/image/ISIC_3662678.jpg
53342,ISIC_3663340,0.0,IP_7198796,data/isic-2020/train-image/image/ISIC_3663340.jpg


In [17]:
print(combined_df.isnull().sum())

isic_id       0
target        0
patient_id    0
file_path     0
dtype: int64


In [18]:
t_max_value = combined_df.shape[0] * (4) * 50 // 16 // 5
t_max_value

133360

In [19]:
skf = StratifiedGroupKFold(n_splits=5)

for fold, ( _, val_) in enumerate(skf.split(combined_df, combined_df.target, combined_df.patient_id)):
      combined_df.loc[val_ , "kfold"] = int(fold)

# Make DataLoader

In [20]:
class ISICDataset_for_Train(Dataset):
    def __init__(self, df, transforms=None):
        self.df_positive = df[df["target"] == 1].reset_index()
        self.df_negative = df[df["target"] == 0].reset_index()
        self.file_names_positive = self.df_positive['file_path'].values
        self.file_names_negative = self.df_negative['file_path'].values
        self.targets_positive = self.df_positive['target'].values
        self.targets_negative = self.df_negative['target'].values
        self.transforms = transforms
        
    def __len__(self):
        return len(self.df_positive) * 2
    
    def __getitem__(self, index):
        if random.random() >= 0.5:
            df = self.df_positive
            file_names = self.file_names_positive
            targets = self.targets_positive
        else:
            df = self.df_negative
            file_names = self.file_names_negative
            targets = self.targets_negative
        index = index % df.shape[0]
        
        img_path = file_names[index]
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        target = targets[index]
        
        if self.transforms:
            img = self.transforms(image=img)["image"]
            
        return {
            'image': img,
            'target': target
        }

    
class ISICDataset(Dataset):
    def __init__(self, df, transforms=None):
        self.df = df
        self.file_names = df['file_path'].values
        self.targets = df['target'].values
        self.transforms = transforms
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        img_path = self.file_names[index]
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        target = self.targets[index]
        
        if self.transforms:
            img = self.transforms(image=img)["image"]
            
        return {
            'image': img,
            'target': target
        }

## Augmentation

In [21]:
data_transforms = {
    "train": A.Compose([
        A.Resize(224, 224),
        A.RandomRotate90(p=0.5),
        A.Flip(p=0.5),
        # Center CROP 넣기
        A.Downscale(p=0.25),
        A.ShiftScaleRotate(shift_limit=0.1, 
                           scale_limit=0.15, 
                           rotate_limit=60, 
                           p=0.5),
        A.HueSaturationValue(
                hue_shift_limit=0.2, 
                sat_shift_limit=0.2, 
                val_shift_limit=0.2, 
                p=0.5
            ),
        A.RandomBrightnessContrast(
                brightness_limit=(-0.1,0.1), 
                contrast_limit=(-0.1, 0.1), 
                p=0.5
            ),
        A.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
        ToTensorV2()], p=1.),
    
    "valid": A.Compose([
        A.Resize(224, 224),
        A.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
        ToTensorV2()], p=1.)
}

# Model Build

## GeM Pooling Layer

In [22]:
class GeM(nn.Module):
    def __init__(self, p=3, eps=1e-6):
        super(GeM, self).__init__()
        self.p = nn.Parameter(torch.ones(1)*p)
        self.eps = eps

    def forward(self, x):
        return self.gem(x, p=self.p, eps=self.eps)
        
    def gem(self, x, p=3, eps=1e-6):
        return F.avg_pool1d(x.clamp(min=eps).pow(p), x.size(-1)).pow(1./p)

## SwinT Model

In [33]:
class ISICSwinTModel(nn.Module):
    def __init__(self, num_classes=1):
        super(ISICSwinTModel, self).__init__()
        self.swin = torchvision.models.swin_v2_b(weights=torchvision.models.Swin_V2_B_Weights.IMAGENET1K_V1)
        self.in_features = self.swin.head.in_features
        self.swin.head = nn.Identity()
        
        self.gem_pooling = GeM()
        self.fc1 = nn.Linear(self.in_features, 512)
        self.bn1 = nn.BatchNorm1d(512)
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, 256)
        self.bn2 = nn.BatchNorm1d(256)
        self.dropout2 = nn.Dropout(0.3)
        self.fc3 = nn.Linear(256, num_classes)

    def forward(self, x):
        features = self.swin(x)
        features = features.unsqueeze(-1)
        pooled_features = self.gem_pooling(features).squeeze(-1)
        x = F.relu(self.bn1(self.fc1(pooled_features)))
        x = self.dropout1(x)
        x = F.relu(self.bn2(self.fc2(x)))
        x = self.dropout2(x)
        return self.fc3(x).squeeze(-1)
    
model = ISICSwinTModel()
model.to(DEVICE)

ISICSwinTModel(
  (swin): SwinTransformer(
    (features): Sequential(
      (0): Sequential(
        (0): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
        (1): Permute()
        (2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      )
      (1): Sequential(
        (0): SwinTransformerBlockV2(
          (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
          (attn): ShiftedWindowAttentionV2(
            (qkv): Linear(in_features=128, out_features=384, bias=True)
            (proj): Linear(in_features=128, out_features=128, bias=True)
            (cpb_mlp): Sequential(
              (0): Linear(in_features=2, out_features=512, bias=True)
              (1): ReLU(inplace=True)
              (2): Linear(in_features=512, out_features=4, bias=False)
            )
          )
          (stochastic_depth): StochasticDepth(p=0.0, mode=row)
          (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
          (mlp): MLP(
            (0): Linear(i

In [50]:
test_input = torch.randn(32, 3, 224, 224).cuda()
output = model(test_input)
print(f"Output Shape: {output.shape}")
print(output)

Output Shape: torch.Size([32])
tensor([-0.5583, -0.1873,  0.2658, -0.5020,  0.1443, -0.0426, -0.7175, -0.1537,
        -0.4800, -0.5520, -0.2102, -0.8281, -0.3493, -0.1937, -0.5204, -0.8315,
         0.4003, -0.1775, -0.0880, -0.0481, -0.8739, -1.1530,  0.3448, -0.5004,
        -0.3193,  0.3377, -0.5796,  0.0099, -0.1258, -0.1345,  0.4063,  0.0279],
       device='cuda:0', grad_fn=<SqueezeBackward1>)


# Train& Validation Code

In [51]:
def train_one_epoch(model, optimizer, scheduler, dataloader, device, epoch):
    model.train()
    
    dataset_size = 0
    running_loss = 0.0
    running_auroc  = 0.0
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, data in bar:
        images = data['image'].to(device, dtype=torch.float)
        targets = data['target'].to(device, dtype=torch.float)
        
        batch_size = images.size(0)
        outputs = model(images).squeeze()
        loss = criterion(outputs, targets)
        loss = loss / 1
            
        loss.backward()
    
        if (step + 1) % 1 == 0:
            optimizer.step()

            # zero the parameter gradients
            optimizer.zero_grad()

            if scheduler is not None:
                scheduler.step()
                
        auroc = binary_auroc(input=outputs.squeeze(), target=targets).item()
        
        running_loss += (loss.item() * batch_size)
        running_auroc  += (auroc * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss / dataset_size
        epoch_auroc = running_auroc / dataset_size
        
        bar.set_postfix(Epoch=epoch, Train_Loss=epoch_loss, Train_Auroc=epoch_auroc,
                        LR=optimizer.param_groups[0]['lr'])
    gc.collect()
    
    return epoch_loss, epoch_auroc

In [52]:
@torch.inference_mode()
def valid_one_epoch(model, dataloader, device, epoch):
    model.eval()
    
    dataset_size = 0
    running_loss = 0.0
    running_auroc = 0.0
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, data in bar:        
        images = data['image'].to(device, dtype=torch.float)
        targets = data['target'].to(device, dtype=torch.float)
        
        batch_size = images.size(0)

        outputs = model(images).squeeze()
        loss = criterion(outputs, targets)

        auroc = binary_auroc(input=outputs.squeeze(), target=targets).item()
        running_loss += (loss.item() * batch_size)
        running_auroc  += (auroc * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss / dataset_size
        epoch_auroc = running_auroc / dataset_size
        
        bar.set_postfix(Epoch=epoch, Valid_Loss=epoch_loss, Valid_Auroc=epoch_auroc,
                        LR=optimizer.param_groups[0]['lr'])   
    
    gc.collect()
    
    return epoch_loss, epoch_auroc

In [53]:
def run_training(model, optimizer, scheduler, device, num_epochs):
    if torch.cuda.is_available():
        print("[INFO] Using GPU: {}\n".format(torch.cuda.get_device_name()))
    
    start = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_epoch_auroc = -np.inf
    history = defaultdict(list)
    
    for epoch in range(1, num_epochs + 1): 
        gc.collect()
        train_epoch_loss, train_epoch_auroc = train_one_epoch(model, optimizer, scheduler, 
                                           dataloader=train_loader, 
                                           device=DEVICE, epoch=epoch)
        
        val_epoch_loss, val_epoch_auroc = valid_one_epoch(model, valid_loader, device=DEVICE, 
                                         epoch=epoch)
    
        history['Train Loss'].append(train_epoch_loss)
        history['Valid Loss'].append(val_epoch_loss)
        history['Train AUROC'].append(train_epoch_auroc)
        history['Valid AUROC'].append(val_epoch_auroc)
        history['lr'].append( scheduler.get_lr()[0] )
        
        # deep copy the model
        if best_epoch_auroc <= val_epoch_auroc:
            print(f"{b_}Validation AUROC Improved ({best_epoch_auroc} ---> {val_epoch_auroc})")
            best_epoch_auroc = val_epoch_auroc
            best_model_wts = copy.deepcopy(model.state_dict())
            PATH = "best_weight.bin".format(val_epoch_auroc, val_epoch_loss, epoch)
            torch.save(model.state_dict(), PATH)
            # Save a model file from the current directory
            print(f"Model Saved{sr_}")
            
        print()
    
    end = time.time()
    time_elapsed = end - start
    print('Training complete in {:.0f}h {:.0f}m {:.0f}s'.format(
        time_elapsed // 3600, (time_elapsed % 3600) // 60, (time_elapsed % 3600) % 60))
    print("Best AUROC: {:.4f}".format(best_epoch_auroc))
    
    # load best model weights
    model.load_state_dict(best_model_wts)
    
    return model, history

## Schedular & Loss

In [28]:
def criterion(outputs, targets):
    return nn.BCEWithLogitsLoss()(outputs, targets)

In [29]:
def fetch_scheduler(optimizer, name):
    if name == 'CosineAnnealingLR':
        scheduler = lr_scheduler.CosineAnnealingLR(optimizer,T_max=t_max_value, 
                                                   eta_min=1e-5)
    elif name == 'CosineAnnealingWarmRestarts':
        scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer,T_0=100, 
                                                             eta_min=1e-6)
    elif name == None:
        return None
        
    return scheduler

In [30]:
def prepare_loaders(df, fold):
    df_train = df[df.kfold != fold].reset_index(drop=True)
    df_valid = df[df.kfold == fold].reset_index(drop=True)
    
    train_dataset = ISICDataset_for_Train(df_train, transforms=data_transforms["train"])
    valid_dataset = ISICDataset(df_valid, transforms=data_transforms["valid"])

    train_loader = DataLoader(train_dataset, batch_size=16, 
                              num_workers=8, shuffle=True, pin_memory=True, drop_last=True)
    valid_loader = DataLoader(valid_dataset, batch_size=32, 
                              num_workers=8, shuffle=False, pin_memory=True)
    
    return train_loader, valid_loader

In [31]:
train_loader, valid_loader = prepare_loaders(combined_df, fold=0)

In [54]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
    
    def forward(self, inputs, targets):
        BCE_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduction='none')
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1-pt)**self.gamma * BCE_loss
        return torch.mean(F_loss)
    
criterion = FocalLoss()

In [55]:
optimizer = optim.AdamW([
    {'params': model.swin.parameters(), 'lr': 1e-5},
    {'params': model.fc1.parameters(), 'lr': 1e-4},
    {'params': model.fc2.parameters(), 'lr': 1e-4},
    {'params': model.fc3.parameters(), 'lr': 1e-3}
], weight_decay=0.01)
scheduler = fetch_scheduler(optimizer, 'CosineAnnealingLR')

In [41]:
model, history = run_training(model, optimizer, scheduler,
                              device=DEVICE,
                              num_epochs=50)

[INFO] Using GPU: NVIDIA GeForce RTX 4090



100%|██████████| 334/334 [00:24<00:00, 13.39it/s, Epoch=1, LR=0.001, Valid_Auroc=0.503, Valid_Loss=0.64] 


Validation AUROC Improved (-inf ---> 0.5034792296820538)
Model Saved



100%|██████████| 334/334 [00:24<00:00, 13.43it/s, Epoch=2, LR=0.001, Valid_Auroc=0.503, Valid_Loss=0.697]


100%|██████████| 334/334 [00:23<00:00, 13.96it/s, Epoch=3, LR=0.001, Valid_Auroc=0.503, Valid_Loss=0.692]


...

Validation AUROC Improved (0.5034879408306772 ---> 0.504041571263429)
Model Saved



100%|██████████| 334/334 [00:24<00:00, 13.53it/s, Epoch=9, LR=0.000996, Valid_Auroc=0.497, Valid_Loss=0.708]


100%|██████████| 334/334 [00:24<00:00, 13.66it/s, Epoch=10, LR=0.000995, Valid_Auroc=0.497, Valid_Loss=0.7]  


 30%|██▉       | 184/616 [00:28<01:06,  6.51it/s, Epoch=11, LR=0.000994, Train_Auroc=0.484, Train_Loss=0.694]


KeyboardInterrupt: 

# Logging

In [ ]:
history = pd.DataFrame.from_dict(history)
history.to_csv("history.csv", index=False)